In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import *

In [4]:
import csv
from datetime import datetime
import os

def load_data(file_path):
    data_dict = dict()

    with open(file_path) as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Numéro du groupe'] != '':
                heures, minutes, _ = map(int, row['TransitTime'].split(':')) 
                total_minutes = heures * 60 + minutes
                data_dict[int(float(row['Numéro du groupe']))] = (
                    int(float(row['Femmes'])),
                    int(float(row['Hommes'])),
                    int(float(row['WCHR'])),
                    total_minutes
                )

    return data_dict

# Load data for each file

file_names = ["5Nov.csv", "7Nov.csv", "22Oct.csv", "24Oct.csv", "23Oct.csv", "30Oct.csv", "DataSeating 2024.csv"]
data_directory = 'data'
# for eachy dictionary : dict[Numéro du groupe : (Femmes, Hommes, WCHR, TransitTime)]
Nov5, Nov7, Oct22, Oct24, Oct23, Oct30, Date0 = [load_data(os.path.join(data_directory, file)) for file in file_names]

In [5]:
Nov5[1][0] # Number of women in group 1 on November 5th

1

In [6]:
Passengers = {}
j = 1

for group_num, data in Nov5.items():
    weights = {'Femmes': 70, 'Hommes': 85, 'WCHR': 100}  # Weight per passenger
    
    # Extracting data
    femmes, hommes, wchr, total_minutes = data
    
    # Enumerating passengers
    for _ in range(femmes + hommes + wchr):
        if femmes > 0:
            weight = weights['Femmes']  
            Passengers[j] = {'gender': 'Femmes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            femmes -= 1
        elif hommes > 0:
            weight = weights['Hommes']
            Passengers[j] = {'gender': 'Hommes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            hommes -= 1
        else:
            weight = weights['WCHR']
            Passengers[j] = {'gender': 'WCHR', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            wchr -= 1
        j += 1

print(Passengers)

{1: {'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}, 2: {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 3: {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 4: {'gender': 'Hommes', 'group': 2, 'weight': 85, 'connection_time': 60}, 5: {'gender': 'Femmes', 'group': 3, 'weight': 70, 'connection_time': 165}, 6: {'gender': 'Hommes', 'group': 3, 'weight': 85, 'connection_time': 165}, 7: {'gender': 'Femmes', 'group': 4, 'weight': 70, 'connection_time': 0}, 8: {'gender': 'Hommes', 'group': 4, 'weight': 85, 'connection_time': 0}, 9: {'gender': 'WCHR', 'group': 4, 'weight': 100, 'connection_time': 0}, 10: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 11: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 12: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 13: {'gender': 'Femmes', 'group': 6, 'weight': 70, 'connection_time': 0}, 14: {'gender': 'Hommes', 'group': 6, 'weig

In [7]:
Passengers[1] # Characteristic of passenger 1 of November 5

{'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}

In [8]:
ranks = 29 #number of rows in the aircraft
n = 90 #number of passengers - TO BE CHANGED
ns=7*ranks #number of seats

def modele_statique():
    m=Model('statique') # Model initialization
    
    # -- Adding variables  --
    # Sij : dict[(int, int) : Var] : Associate passenger i with seat j
    S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, ns+1)}

    # -- Adding constraints  --
    
    # Max. 1 seat per passenger
    for j in range (1, ns+1):
        m.addConstr(sum(S[(i,j)] for i in range (1, n+1)) <=1, name="PassengerMax")

    #Each passenger has one and only one seat
    for i in range (1, n+1):
        m.addConstr(sum(S[(i,j)] for j in range (1, ns+1)) == 1, name="SeatMax")

    #The central aisle is left free
    for i in range (1, n+1):
        m.addConstr(S[(i, 4)] == 0)

    #Barycenter

    return m, S

In [9]:
## Fonctions objectifs

m,S=modele_statique()

#Les passagers en transit sont placés à l'avant de l'avion

def obj_transit():
    T= {} 
    
    
    for passenger in Passengers:
        if Passengers[passenger]['connection_time'] >0:   #passager en transit
            T[passenger]=Passengers[passenger]['connection_time']

    #T=sorted(T.items(), key=lambda item:item[1], reverse=True)
    P=list(T.keys())  #retourne la liste des identifiants des passagers en transit
    

    f=0
    for k in range (len(P)):
        for j in range (1, ns+1):
            q=(j-1)//7
            f+= S[(P[k],j)] * (1/T[P[k]]) * q
            

    return f

m.setObjective(obj_transit(), GRB.MINIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01


In [10]:
# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update() 

# -- Affichage en mode texte du PL --
display(m)

<gurobi.Model MIP instance statique: 383 constrs, 18270 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [11]:
# -- Résolution --
m.optimize()

# -- Affichage de la solution --
print("Les places dans l'avion sont les suivantes :", [(i,j) for i in range(1,n+1) for j in range (1,ns+1) if S[(i,j)].x])

Les places dans l'avion sont les suivantes : [(1, 187), (2, 153), (3, 123), (4, 2), (5, 25), (6, 21), (7, 139), (8, 69), (9, 188), (10, 131), (11, 136), (12, 92), (13, 185), (14, 64), (15, 10), (16, 19), (17, 96), (18, 7), (19, 3), (20, 8), (21, 5), (22, 134), (23, 142), (24, 173), (25, 122), (26, 162), (27, 89), (28, 99), (29, 158), (30, 178), (31, 57), (32, 1), (33, 55), (34, 24), (35, 93), (36, 107), (37, 120), (38, 161), (39, 193), (40, 113), (41, 31), (42, 144), (43, 203), (44, 88), (45, 29), (46, 53), (47, 73), (48, 100), (49, 147), (50, 11), (51, 13), (52, 9), (53, 104), (54, 116), (55, 40), (56, 119), (57, 168), (58, 180), (59, 91), (60, 137), (61, 23), (62, 95), (63, 71), (64, 197), (65, 39), (66, 18), (67, 15), (68, 102), (69, 45), (70, 14), (71, 6), (72, 202), (73, 36), (74, 51), (75, 176), (76, 84), (77, 90), (78, 83), (79, 20), (80, 43), (81, 115), (82, 130), (83, 148), (84, 67), (85, 12), (86, 16), (87, 17), (88, 179), (89, 110), (90, 72)]
